# Обратная задача кинематики PUMA

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
from IPython.display import HTML
%matplotlib notebook

In [ ]:
from kinematics import Vector, Quaternion, Transform
import graphics

## Рассмотрим кинематическую модель, более приближенную к реальности

Возьмем манипулятор ABB IRB 140.

![irb 140](fig/irb_140.png)

Расстояние | Значение (мм)
-----------|--------------
Высота колонны | 352
Вынос вперед | 70
Плечо | 360
Предплечье | 380
Кисть | 65

In [ ]:
irb_l = [352, 70, 350, 380, 65]

Учтем диапазон изменения обобщенных координат (град)

Обобщенная координата | Минимум | Максимум
----------------------|---------|---------
A1 | -180 | 180
A2 | -110 | 90
A3 | -140 | 140
A4 | -180 | 180
A5 | -115 | 115
A6 | -180 | 180

In [ ]:
irb_lim = [
    (-180, 180),
    (-90, 110),
    (-230, 50),
    (-200, 200),
    (-115, 115),
    (-400, 400)
]

При решении прямой задачи учтем что физические нули расположены неочевидно

In [ ]:
def irb_chain(q, l):
    base = Transform.identity()
    column = base + Transform(
        Vector(0, 0, l[0]),
        Quaternion.from_angle_axis(q[0], Vector(0, 0, 1))
    )
    shoulder = column + Transform(
        Vector(l[1], 0, 0),
        Quaternion.from_angle_axis(q[1], Vector(0, -1, 0))
    )
    elbow = shoulder + Transform(
        Vector(0, 0, l[2]),
        Quaternion.from_angle_axis(q[2], Vector(0, 1, 0))
    )
    wrist = elbow + Transform(
        Vector(l[3], 0, 0),
        Quaternion.from_angle_axis(q[3], Vector(1, 0, 0)) *
        Quaternion.from_angle_axis(q[4], Vector(0, 1, 0))
    )
    flange = wrist + Transform(
        Vector(l[4], 0, 0),
        Quaternion.from_angle_axis(q[5], Vector(1, 0, 0)) *
        Quaternion.from_angle_axis(np.pi / 2, Vector(0, 1, 0))
    )
    return [
        base,
        column,
        shoulder,
        elbow,
        wrist,
        flange
    ]

Решим обратную задачу кинематики:

In [ ]:
def irb_ik(target, l, i=[1, 1, 1]):
    wrist = target + Vector(0, 0, -l[4]) +  Vector(0, 0, -l[0])
    projection = Vector(wrist.x, wrist.y, 0)
    q0 = Vector(0, 1, 0).angle_to(projection, Vector(0, 0, 1)) - np.pi / 2 * i[0] + np.pi
    d = ((projection.magnitude() - i[0] * l[1]) ** 2 + wrist.z ** 2) ** 0.5
    q2 = -i[1] * np.arccos(
        (l[2] **  2 + l[3] ** 2 - d ** 2) /\
        (2 * l[2] * l[3])
    ) + np.pi / 2
    triangle_angle = np.arcsin(
        l[3] * i[0] * np.sin(q2 - np.pi / 2) / d
    )
    lift_angle = np.arctan2(
        wrist.z,
        (projection.magnitude() - i[0] * l[1])
    )
    q1 = -i[0] * (np.pi / 2 + triangle_angle - lift_angle)
    ori = Quaternion.from_angle_axis(q0, Vector(0, 0, 1)) *\
        Quaternion.from_angle_axis(q1, Vector(0, -1, 0)) *\
        Quaternion.from_angle_axis(q2, Vector(0, 1, 0))
    ez = ori * Vector(1, 0, 0)
    ey = ori * Vector(0, 1, 0)
    tz = target.rotation * Vector(0, 0, 1)
    ty = target.rotation * Vector(0, 1, 0)
    wy = ez.cross(tz)
    q3 = ey.angle_to(wy, ez) + np.pi / 2 - np.pi / 2 * i[2]
    q4 = ez.angle_to(tz, wy) * i[2]
    q5 = wy.angle_to(ty, tz) + np.pi / 2 -np.pi / 2 * i[2]
    return [q0, q1, q2, q3, q4, q5]

Зададим закон изменения положения:

In [ ]:
def target(t, total):
    return Transform(
        Vector(0, 500, 1000 * t / total) if t / total < 0.5 else Vector((t / total - 0.5) * 500, 500, 500),
        Quaternion.from_angle_axis(
            t / total * np.pi + np.pi if t / total < 0.5 else 3 / 2 * np.pi,
            Vector(1, 0, 0)
        )
    )

Укажем флаги конфиругации:

In [ ]:
irb_i = [1, 1, 1]

In [ ]:
(x, y, z) = graphics.chain_to_points(
    irb_chain([0, 0, 0, 0, 0, 0], irb_l)
)
fig, ax = graphics.figure(1000)
lines, = ax.plot(x, y, z, color="#000000")
rt, gt, bt = graphics.axis(ax, Transform.identity(), 1)
rf, gf, bf = graphics.axis(ax, Transform.identity(), 1)

total = 100

def animate(frame):
    t = target(frame, total)
    q = irb_ik(
        t,
        irb_l,
        irb_i
    )
    chain = irb_chain(q, irb_l)
    (x, y, z) = graphics.chain_to_points(chain)
    lines.set_data_3d(x, y, z)
    global rt, gt, bt, rf, gf, bf
    rt.remove(); gt.remove(); bt.remove(); rf.remove(); gf.remove(); bf.remove()
    rt, gt, bt = graphics.axis(ax, t, 100)
    rf, gf, bf = graphics.axis(ax, chain[-1], 100)


animate(0)
fps = 25
irb_ani = animation.FuncAnimation(
    fig,
    animate,
    frames=total,
    interval=1000.0/fps
)